In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
import xarray as xr
from  lib.advection import xr_vert_adv
from lib.thermo import liquid_water_temperature, get_dz

In [ ]:
stat = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/stat.nc")
fields_3d = xr.open_mfdataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/3d/*.nc")
fields_2d = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc")

# for some reason the time values are all scrambled up.
fields_2d = fields_2d.isel(time=fields_2d.time.values.argsort())
data = xr.merge((fields_2d, fields_3d), join='inner').isel(y=slice(24, 40))

rho = stat.RHO[0]

data  = data.assign(
    sl=liquid_water_temperature(data.TABS, data.QN, data.QP)
)


In [ ]:
div = data.U.diff('x')/160000 + data.V.diff('y')/160000 + 

In [ ]:
W = (-div*rho*dz).cumsum('z')/rho

In [ ]:
W.isel(x=0, y=8).T.plot()

In [ ]:
dz = get_dz(rho.z)

In [ ]:
loc = data.isel(x=0, y=8).compute()

In [ ]:
V = xr_vert_adv(loc.W, loc.sl, rho)

In [ ]:
(-V).T.plot()

In [ ]:
V_center = (loc.W * loc.sl * rho).diff('z')/rho/dz

In [ ]:
V_center.T.plot()

In [ ]:
(loc.sl.centderiv('z') * loc.W).plot()